In [ ]:
!pip3 install -U --user pip numpy pandas matplotlib rich

In [10]:
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
from rich import console
from rich import progress

BASE_DIR = Path.cwd().resolve()
DATA_DIR = BASE_DIR / "Stocks"
RESULTS_DIR = BASE_DIR / "results"
STOCKS_OBJECT = BASE_DIR / "preprocessed.o"
STOCKS_RAW_OBJECT = BASE_DIR / "raw.o"
HEADERS = ["date", "open", "high", "low", "close", "volume", "openint"]


def load_raw_stocks():
    if STOCKS_RAW_OBJECT.exists():
        print("Loading raw stocks...")
        with open(STOCKS_RAW_OBJECT, "rb") as stream:
            return pickle.load(stream)
    stocks = []
    progress_bar = progress.Progress(
        "{task.description}",
        progress.BarColumn(),
        "{task.completed} of {task.total} ({task.percentage:>6.2f}%)",
        progress.TimeRemainingColumn(),
        progress.TimeElapsedColumn(),
    )
    with progress_bar:
        for stock in progress_bar.track(
            list(sorted(DATA_DIR.iterdir())), description="Loading stocks..."
        ):
            stock_name = stock.name.split(".", 1)[0]
            df = pd.read_csv(stock, header=0, names=HEADERS)
            # Empty CSV.
            if df.size == 0:
                continue
            df["date"] = pd.to_datetime(df["date"])
            df["open"] = df["open"].astype(np.float64)
            df["high"] = df["high"].astype(np.float64)
            df["low"] = df["low"].astype(np.float64)
            df["close"] = df["close"].astype(np.float64)
            df["volume"] = (df["volume"] * 0.1).astype(np.uint32)
            df["stock"] = stock_name
            df = df[["date", "stock", "open", "low", "high", "close", "volume"]]
            stocks.append(df)
    with console.Console().status("Merging stocks..."):
        stocks = pd.concat(stocks, ignore_index=True)
        stocks = stocks.set_index(["date", "stock"]).sort_index()
    with open(STOCKS_RAW_OBJECT, "wb") as stream:
        pickle.dump(stocks, stream)
    return stocks


def load_stocks(force: bool = False):
    if STOCKS_OBJECT.exists() and not force:
        print("Loading stocks...")
        with open(STOCKS_OBJECT, "rb") as stream:
            return pickle.load(stream)
    stocks = []
    progress_bar = progress.Progress(
        "{task.description}",
        progress.BarColumn(),
        "{task.completed} of {task.total} ({task.percentage:>6.2f}%)",
        progress.TimeRemainingColumn(),
        progress.TimeElapsedColumn(),
    )
    with progress_bar:
        for stock in progress_bar.track(
            list(sorted(DATA_DIR.iterdir())), description="Loading stocks..."
        ):
            stock_name = stock.name.split(".", 1)[0]
            df = pd.read_csv(stock, header=0, names=HEADERS)
            # Empty CSV.
            if df.size == 0:
                continue
            df["date"] = pd.to_datetime(df["date"])
            df["buy-open_sell-high"] = ((df["high"] * 0.99) / (df["open"] * 1.01)).replace(
                [-np.inf, np.inf], np.nan
            )
            df["buy-low_sell-close"] = ((df["close"] * 0.99) / (df["low"] * 1.01)).replace(
                [-np.inf, np.inf], np.nan
            )
            df["buy"] = np.where(
                df["buy-open_sell-high"] > df["buy-low_sell-close"],
                df["open"] * 1.01,
                df["low"] * 1.01,
            )
            df["sell"] = np.where(
                df["buy-open_sell-high"] > df["buy-low_sell-close"],
                df["high"] * 0.99,
                df["close"] * 0.99,
            )
            df["winratio"] = np.where(
                df["buy-open_sell-high"] > df["buy-low_sell-close"],
                df["buy-open_sell-high"],
                df["buy-low_sell-close"],
            )
            df["buy-action"] = np.where(
                df["buy-open_sell-high"] > df["buy-low_sell-close"],
                "buy-open",
                "buy-low",
            )
            df["sell-action"] = np.where(
                df["buy-open_sell-high"] > df["buy-low_sell-close"],
                "sell-high",
                "sell-close",
            )
            df["winratio"] = df["winratio"].replace([-np.inf, np.inf], np.nan)
            df = df[df["winratio"] >= 1.01]
            if df.size == 0:
                continue
            df["buy"] = df["buy"].astype(np.float64)
            df["sell"] = df["sell"].astype(np.float64)
            df["volume"] = (df["volume"] * 0.1).astype(np.uint32)
            df["winratio"] = df["winratio"].astype(np.float64)
            df["stock"] = stock_name
            df = df[
                ["date", "stock", "buy", "sell", "volume", "winratio", "buy-action", "sell-action"]
            ]
            stocks.append(df)
    with console.Console().status("Merging stocks..."):
        stocks = pd.concat(stocks, ignore_index=True)
        stocks = stocks.set_index(["date", "stock"]).sort_index()
    with open(STOCKS_OBJECT, "wb") as stream:
        pickle.dump(stocks, stream)
    return stocks


def solve_small(raw_stocks: pd.DataFrame, stocks: pd.DataFrame):
    first: pd.Timestamp
    last: pd.Timestamp
    dates = raw_stocks.sort_index().index.get_level_values("date").values
    first, last = dates[0], dates[-1]
    print(first, last)
    timestamp: pd.Timestamp
    output = BASE_DIR / "small.txt"
    balance = 1.0
    n = 0
    actions = []
    progress_bar = progress.Progress(
        "{task.description}",
        progress.BarColumn(),
        "{task.completed} of {task.total} ({task.percentage:>6.2f}%)",
        progress.TimeRemainingColumn(),
        progress.TimeElapsedColumn(),
    )
    daystocks: pd.DataFrame
    with progress_bar:
        for timestamp, daystocks in progress_bar.track(
            list(stocks.groupby(by=["date"])), description="Solving small..."
        ):
            winnings = 0
            least_winnings = 1e7 if balance > 1e8 else balance * 2 ** -(1e-8 * balance)
            daystocks = daystocks.droplevel("date")
            daystocks["buyable"] = balance // daystocks["buy"]
            daystocks["volume"] = daystocks[["volume", "buyable"]].min(axis=1).astype(np.uint32)
            daystocks = daystocks[daystocks["volume"] > 0]
            daystocks["winnings"] = (
                daystocks["volume"] * (daystocks["sell"] - daystocks["buy"])
            ).astype(np.float64)
            daystocks = daystocks[daystocks["winnings"] >= least_winnings]
            if daystocks.size == 0:
                continue
            daystocks = daystocks.sort_values("winnings", ascending=False).head(1)
            for stock_name, stock in daystocks.iterrows():
                n += 2
                buy, sell, volume = stock["buy"], stock["sell"], stock["volume"]
                buy_action, sell_action = stock["buy-action"], stock["sell-action"]
                balance -= buy * volume
                winnings += sell * volume
                actions.append([timestamp, buy_action, stock_name, volume])
                actions.append([timestamp, sell_action, stock_name, volume])
            balance += winnings
            if n == 1000:
                break
    actions = pd.DataFrame(actions, columns=["date", "action", "stock", "volume"])
    actions["action"] = pd.Categorical(
        actions["action"],
        categories=["buy-open", "buy-low", "sell-high", "sell-close"],
        ordered=True,
    )
    actions = actions.sort_values(by=["date", "action"])
    actions["stock"] = actions["stock"].str.upper()
    with open(output, "w") as stream:
        stream.write(f"{actions.shape[0]}\n")
        actions.to_csv(stream, sep=" ", header=False, index=False, date_format="%Y-%m-%d")
    print(n, balance)
    return balance, actions


def solve_large(raw_stocks: pd.DataFrame, stocks: pd.DataFrame):
    timestamp: pd.Timestamp
    output = BASE_DIR / "large.txt"
    balance = 1.0
    n = 0
    actions = []
    progress_bar = progress.Progress(
        "{task.description}",
        progress.BarColumn(),
        "{task.completed} of {task.total} ({task.percentage:>6.2f}%)",
        progress.TimeRemainingColumn(),
        progress.TimeElapsedColumn(),
    )
    daystocks: pd.DataFrame
    with progress_bar:
        for timestamp, daystocks in progress_bar.track(
            list(stocks.groupby(by=["date"])), description="Solving large..."
        ):
            winnings = 0
            daystocks = daystocks.droplevel("date")
            daystocks["buyable"] = balance // daystocks["buy"]
            daystocks["volume"] = daystocks[["volume", "buyable"]].min(axis=1).astype(np.uint32)
            daystocks = daystocks[daystocks["volume"] > 0]
            if daystocks.size == 0:
                continue
            daystocks["winnings"] = (
                daystocks["volume"] * (daystocks["sell"] - daystocks["buy"])
            ).astype(np.float64)
            daystocks = daystocks.sort_values(["winnings"], ascending=False).head(80)
            for stock_name, stock in daystocks.iterrows():
                buy, sell, volume = stock["buy"], stock["sell"], stock["volume"]
                buy_action, sell_action = stock["buy-action"], stock["sell-action"]
                if balance < buy:
                    continue
                n += 2
                buyable = int(min(volume, balance // buy))
                balance -= buy * buyable
                winnings += sell * buyable
                actions.append([timestamp, buy_action, stock_name, buyable])
                actions.append([timestamp, sell_action, stock_name, buyable])
                if n == 1000000:
                    break
            balance += winnings
            if n == 1000000:
                break
    actions = pd.DataFrame(actions, columns=["date", "action", "stock", "volume"])
    actions["action"] = pd.Categorical(
        actions["action"],
        categories=["buy-open", "buy-low", "sell-high", "sell-close"],
        ordered=True,
    )
    actions = actions.sort_values(by=["date", "action"])
    actions["stock"] = actions["stock"].str.upper()
    with open(output, "w") as stream:
        stream.write(f"{actions.shape[0]}\n")
        actions.to_csv(stream, sep=" ", header=False, index=False, date_format="%Y-%m-%d")
    print(n, balance)
    return balance, actions

In [11]:
raw_stocks = load_raw_stocks()
stocks = load_stocks()
solve_small(raw_stocks, stocks)

Loading raw stocks...
Loading stocks...
1962-01-02T00:00:00.000000000 2017-11-10T00:00:00.000000000


Output()

1000 14994859581.417542


(14994859581.417542,
           date      action stock    volume
 0   1974-12-04     buy-low   WMT       120
 1   1974-12-04  sell-close   WMT       120
 2   1989-03-22     buy-low  DWDP         2
 3   1989-03-22  sell-close  DWDP         2
 4   1989-06-20    buy-open  DWDP        19
 ..         ...         ...   ...       ...
 995 2013-05-23  sell-close   HPQ  21801526
 996 2013-05-24    buy-open   VRX   1038361
 997 2013-05-24   sell-high   VRX   1038361
 998 2013-05-28     buy-low  TSLA   1966185
 999 2013-05-28  sell-close  TSLA   1966185
 
 [1000 rows x 4 columns])

In [12]:
solve_large(raw_stocks, stocks)

Output()

957522 296449471848.97797


(296449471848.97797,
              date      action stock  volume
 0      1962-01-08     buy-low    GE       1
 1      1962-01-08  sell-close    GE       1
 2      1962-01-31     buy-low    GE       1
 3      1962-01-31  sell-close    GE       1
 4      1962-05-01     buy-low    GE       1
 ...           ...         ...   ...     ...
 957499 2017-11-10  sell-close   HCC  214328
 957501 2017-11-10  sell-close    QD  315623
 957503 2017-11-10  sell-close   TCO  164973
 957507 2017-11-10  sell-close   UAA  786513
 957511 2017-11-10  sell-close   EFX  216070
 
 [957522 rows x 4 columns])